# Optiver Realized Volatility Prediction
# Introduction to financial concepts and data
# DIscottionで最もvoteされているカーネル

スコアは高くないが、導入として読むことにする  
Optiverのデータサイエンティストによるチュートリアルノートブック

[リンク](https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data)

# オーダーブック

買いと売りの価格による日本でいうところの板。  
密度が高いものは頻繁にトレードされているか、動きが激しいとみなせる。  

## 疑問
状態を示すデータになるはず。更新タイミングはデータの初めか何かだろうか??  


# トーレド

実際に売買が行われたあとのオーダーブックとの差。  
というより、トレードによって上記オーダーブックの値が変化することになる。  
日本と同じで、最低売値から買われる。  

下のオーダーブックの場合、  
価格 : 148、　数量 : 20  
のトーレドが行われたと推測できる。

![Imgur](https://i.imgur.com/16Qt255l.png)
![Imgur](https://i.imgur.com/jTuN081l.png)

# Market making and market efficiency

![Imgur](https://i.imgur.com/f5TOZJCl.png)

上の表の市場の場合流動性(効率とも呼ばれてる)は低いといえる。  
価格 : 148  
では買えず、待ちの状態になる。  
価格 : 問わない, 数量 : 20 の注文を出した場合  
149 : 1  
150 : 12  
151 : 7  
となり、希望通りの取引はできない。これを非効率的な市場と呼ぶ。  
これを解決させるのが、***マーケットメーカー***と呼ばれる存在になる。  
金融機関が流動性確保のために、こういった株に対して売買気配置を示すこと。（国からの命令）  
これによって一定の流動性が保たれる。

# オーダーブック統計

Optiverのデータサイエンティスト推奨のオーダブックから抽出できる統計値。  

## bid/ask spread

最高売値と最低買値の比率をとって計算される値。  
小さいと流動性が高いと見なせる。  

$$BidAskSpread = BestOffer/BestBid -1$$  

## Weighted averaged price
日本語の加重平均価格  
100円のケーキと200円のケーキを1個ずつ買えば、平均は150円。では100円のケーキを7個と200円のケーキを3個買えば、（100円×7個）＋（200円×3個）＝1,300円。これを10個で割ったら130円になる。  
これを先程のオーダーブックに適用したものになる。  

$$WAP = \frac{BidPrice_{1}*AskSize_{1} + AskPrice_{1}*BidSize_{1}}{BidSize_{1} + AskSize_{1}}$$

## 株式評価額

加重平均が同じな場合でも、株式評価額は異なる。  
というのも、オファーサイズが大きい場合その市場には売り手が多く存在することになる。そうなると一般的に株の価値は上がりづらく結果として株式評価額は小さくなる。


In [ ]:
# 上のWAPを計算

bidsize = 251
asksize = 221
bidprice = 147
askprice = 148

(bidprice * asksize + askprice * bidsize) / (bidsize + asksize)

# Log retuen

株価の割合の比率を対数で取ったもの。  
$$r_{t_1, t_2} = \log \left( \frac{S_{t_2}}{S_{t_1}} \right)$$
$S_t $ : ある時間$t$での株価格$S$

基本的には10分が目安となる。また
- 時間をまたいで加法的である $r_{t1},r_{t2}+r_{t2},r_{t3}=r_{t1},r_{t3} $
- 通常のリターンは-100%以下にはならないが、Log returnは境界がない

Logの図  
価格が落ちた場合、rは少数になるのでyは-の値を取る。  
価格が上がった場合.rは自然数におなるのでyは+の値を取る。  
もし最終的に変化がなければ、0を取る。

![Imgur](https://i.imgur.com/PieGDUqh.png)

# Realized volatility

Log returnの標準偏差を１年間とみなして計算する年間標準偏差をボラティリティとよぶ。  
今回の目的変数。  
$$\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}$$
10分刻みでLog returnを求めて、それを一年単位で行い標準偏差を求める。  

シンプルなLog returnとは違い、標準偏差になるので最終変動が0であっても10分刻みで動きが激しければボラティリティは高い値をとる。  
つまり流動性が高く、いい市場になっているといえる。

# Competition data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train.head()

In [ ]:
# time_idが5の時のstock_id=0の取引データ

book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

In [ ]:
# オーダーブック

book_example.head()

- time_id  
    時間id,名前は時間には関係ないので注意  
- seconds_in_bucket  
    秒数, 0から絶対始まる。動きがあると行が追加される。  
- bid_size1, bid_size2  
    競争力の高い買いの株式数  
    和訳が難しいが、現在価格に近い買い株式数の事  
- ask_size1, ask_size2  
    同様に売りの株式数  
- bid_price1, ask_price1, bid_price2, ask_price2   
    最小の買値/ 現在の価格と最小から２番目の買値/ 現在の価格  
    最大の売値 / 現在の価格と最大から2番目の売値 / 現在の価格  
- stock_id  
    銘柄のid

In [ ]:
# トレード履歴

trade_example.head()

- price  
    1秒間に取引が成立した平均価格。  
    価格は取引が成立した株式数と金額の平均値で加重平均を取ったもので正規化されている
- size  
    priceの価格でいくつの株式数が取引されたか  
- order_count  
    取引注文の数

## Realized volatility calculation
### 1秒あたりの移動平均
$$WAP = \frac{BidPrice_{1}*AskSize_{1} + AskPrice_{1}*BidSize_{1}}{BidSize_{1} + AskSize_{1}}$$

In [ ]:
# stock_id = 0の銘柄の移動平均を図示  
# 横軸は1秒(second_in_bucket)

book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
# book_exsampleに上で作成した移動平均の差分(diff())の列を作成
book_example.loc[:,'log_return'] = log_return(book_example['wap'])

# 取引が行われていない秒数は欠損で入るため、　Flaseのvalueを消して再代入
book_example = book_example[~book_example['log_return'].isnull()]

In [ ]:
book_example.head()

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

### ボラティリティ(目的変数)の求め方

In [ ]:
# log_returnの二乗のルートがボラティリティになるので求める関数
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

# 今回のオーダーブックに適用
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

## 予測
ボラティリティについて一般的に知られている事実は、自己相関する傾向がある。この特性を利用して、最初の10分間の実現ボラティリティーを使用して、実現ボラティリティーを「予測」するナイーブモデルを実装する  
また、メモリ削減のため各銘柄(stock_id)毎に計算する必要がある。それをまとめて、提出を行う。

In [ ]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('../input/optiver-realized-volatility-prediction/book_train.parquet/*')

In [ ]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    # fileをしていしてdataframeに
    df_book_data = pd.read_parquet(file_path)
    
    # 移動平均を計算
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    
    # time_id毎にdataframeを書き換える、その後log_retuenの計算結果をシリーズとして得てdataframeに追加する。
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    
    # null(変化がない)行を削除
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    
    # time_id毎のボラティリティを算出し、time_idにindexを紐づける
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    
    # 第二引数の名前にボラティリティを求めた列の名前を変更
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    
    # path名の後ろがstock_idなのでそこから名前を得る
    stock_id = file_path.split('=')[1]
    
    # 名前をstock_id - time_id に変更
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    # 二列をdataframeとして返す
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    # 空dataframeを作成
    df_past_realized = pd.DataFrame()
    
    # 上で作ったものに追加
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
        
    return df_past_realized

In [ ]:
# 全データでdfを作成
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [ ]:
df_past_realized_train.head()

In [ ]:
df_past_realized_train.info()

In [ ]:
# trainファイルにrow_idの列を追加
# predの列を追加
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

trainはparquetと呼ばれる形で格納されている。  
中身はこんな感じ  
![Imgur](https://i.imgur.com/KHv7zq1l.png)

In [ ]:
df_joined.head()

In [ ]:
# RMSPEとR^2を計算

from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

## 考察
trainに入っているデータはstock_idに対して600秒(10分しかない)  
このデータで　次に来る600秒のボラティリティを予測するというのが今回の目的になる。  
上のスコアは前の600秒のボラティリティをそのまま次の600秒のボラティリティとして予測した際のスコアになる。

In [ ]:
# 提出方法

list_order_book_file_test = glob.glob('../input/optiver-realized-volatility-prediction/book_test.parquet/*')
df_naive_pred_test = past_realized_volatility_per_stock(list_file=list_order_book_file_test,
                                                           prediction_column_name='target')
df_naive_pred_test.to_csv('submission.csv',index = False)

# 最後に
実際に提出した際のパブリックリーダーボードの値は過去の市場のデータとなるが、今回予測するのは未来のデータになるのでプライベートリーダーボードの値とは大きく異なる。  
パブリックでいいスコアが出ても大きな意味をなさない....  
というかtestには3つのデータしか入っておらず、実際に提出してみてのスコアをみる必要があるが、どの程度のデータ量で検証されているのだろう....  
過学習抑制が大きな問題になりそう

# このカーネルについていたコメント

## 移動平均について
bid*bid + ask*ask / ask + bit　じゃないの？  
買い手が売り手よりも多い場合（bid_size >> ask_size）、価格が上昇することが予想され、買い手が多ければ多いほど、WAPはask_priceに近づくとされている。  
もしかしたら特徴量にできるかも

## time_idについて

複数の10分間の市場データを使用して、予測を立てても良いのか。  
time_idは時間とリンクしていないので同time_idでないと予測は不可能  